<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0], 'GPU')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Content_Project/data.csv')

In [5]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [6]:
df.dropna(inplace=True)

In [7]:
df1=df[:500]

In [8]:
df1.to_csv('data1.csv',index=False)

In [9]:
data=pd.read_csv('data1.csv')

In [10]:
data.shape

(500, 6)

**Split the dataset into train and evaluate**

In [11]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2)

In [12]:
train_data.shape

(400, 6)

In [13]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')


In [14]:
import csv

# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [15]:
# import csv

# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [16]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [18]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [19]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)



/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [20]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


In [21]:
pip install pytorch-lightning[callbacks]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from pytorch_lightning.callbacks import ModelCheckpoint

class MyModelCheckpoint(ModelCheckpoint):
    def on_init_end(self, trainer):
        super().on_init_end(trainer)

In [23]:
#model checkpoint
checkpoint_callback = MyModelCheckpoint(
    dirpath='./checkpoints',
    filename='checkpoint-{epoch:02d}',
    save_top_k=3,
    monitor='loss',
    mode='min'
)


In [24]:
# Set up the trainer with the appropriate training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_total_limit=2,
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,
   # callbacks=[checkpoint_callback]
)



In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained model and tokenizer
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Define your custom prompt
prompt = "Artificial intelligence is"

# Define the maximum length of each segment
max_segment_length = 1024

# Define the total length of the final article
total_length = 5000

# Generate the first segment of text
input_ids = tokenizer.encode(prompt, return_tensors='pt')
generated_text = model.generate(
    input_ids=input_ids,
    max_length=max_segment_length,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Decode the generated text and print it
generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(generated_text)

# Generate additional segments of text
while len(generated_text) < total_length:
    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    # Generate the next segment of text
    next_segment = model.generate(
        input_ids=input_ids,
        max_length=max_segment_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    
    # Decode the generated text and append it to the previous segment
    next_segment = tokenizer.decode(next_segment[0], skip_special_tokens=True)
    generated_text += next_segment

# Print the final generated text
print(generated_text)


In [25]:
# Fine-tune the model on your dataset
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.507422
2,No log,3.482466
3,3.444200,3.478778
4,3.444200,3.480862
5,3.444200,3.485349


TrainOutput(global_step=855, training_loss=3.318068690606725, metrics={'train_runtime': 517.7165, 'train_samples_per_second': 26.366, 'train_steps_per_second': 1.651, 'total_flos': 891659059200000.0, 'train_loss': 3.318068690606725, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.298492670059204,
 'eval_runtime': 39.8979,
 'eval_samples_per_second': 92.937,
 'eval_steps_per_second': 11.63,
 'epoch': 5.0}

In [ ]:
trainer

In [26]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import torch

In [35]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy?) for just $99. And then, with the exception of the base model, only $34 when it comes to hardware. The iPhone 5 is coming in two models, one with $799 in cash and one with $999 in cash, but with both starting at $1,599 in cash and $999 in cash.  The iPhone SE has $999 and $999. The iPhone 12 is coming in two versions with the base model and the entry-level models with the entry-level model with the entry-level model. If you’re new to the line of iPhones and want to upgrade in price, this is your option. If you’ve made it that far in your life and have a budget, this isn’t the phone for you.  Buy your iPhone SE  and the iPhone SE   are the best and the best phones for you in the long run. The  iPhone  is not


In [ ]:
 # Tokenize the prompt and convert it to input ids
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    # Generate the next segment of text
    outputs = model.generate(input_ids=input_ids, max_length=max_segment_length, do_sample=True)
    segment = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Append the segment to the generated text
    generated_text += segment
    
    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]

In [34]:

# Generate text using the fine-tuned model
prompt = 'Artificial Intelligence'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))


# Define the total length of the final article
total_length = 5000
max_segment_length=1024

#print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

# Generate additional segments of text
while len(generated_text) < total_length:
    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt =generated_text[-max_segment_length:]
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    
    # Generate the next segment of text
    next_segment = model.generate(
        input_ids=input_ids,
        max_length=max_segment_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    
    # Decode the generated text and append it to the previous segment
    next_segment = tokenizer.decode(next_segment[0], skip_special_tokens=True)
    generated_text += next_segment

# Print the final generated text
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial Intelligence and its future, “will continue to evolve in the near and distant future to enable a whole new type of computing world that will revolutionize the way our society operates.” The event was organized by  the Stanford Artificial Intelligence Association. It also included a virtual networking event called the Intercom Summit that brought together experts from the AI community, startups, researchers, and technologists.
Uber is set to introduce its new service, Voucher, to its U.S. fleets beginning this fall. A Voucher ride-hailing service will launch at the start of 2019, but Uber says it will have its first fleet by late 2021. The company has so far declined to comment on when Voucher will be available in the U.S. — and the exact date for the rollout remains to be determined. The firm initially began developing a service through Uber in the U.S., but found its business in Southeast Asia and the Middle East. The product


ValueError: ignored

In [50]:


# Set the maximum length of each segment
max_segment_length = 1024

# Set the initial prompt
prompt = "Artificial intelligence"

# Initialize the generated text
generated_text = prompt

# Generate segments of text and stitch them together
while len(generated_text) < 500:
    # Encode the prompt as input IDs
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate the next segment of text
    output = model.generate(input_ids=input_ids.to('cuda'),
                             max_length=max_segment_length,
                             do_sample=True,
                             top_p=0.95,
                             top_k=50)
    
    # Decode the output to text
    segment = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the segment to the generated text
    generated_text += segment

    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]

# Print the generated text
print(generated_text)
final_text=generated_text


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligenceArtificial intelligence has taken a big step toward giving us the tools and technologies to solve a number of  fundamental problems for society today.  It’s now clear from the start that AI is about making sense of what’s going on within the context of what’s going on at home, as well. And today’s revelations have added even more fuel to that fire. AI is also increasingly being used and described by various  corporate  companies and governments  to manipulate and exploit the general population without anyone’s knowledge. While the world is still reeling in the devastation caused by pandemic, it’s not the  rest of the world that hasn’t started noticing the COVID-19 trend. At the same time, as I understand, the pandemic has already killed tens of thousands of people and caused millions of dollars in social unrest that has taken root in other parts of the world, all with the explicit aim of reducing the spread of disease to all of us. The same is true of other aspec

In [39]:
import requests
import json

# Set up API endpoint and headers
url = "https://text-monkey-summarization.p.rapidapi.com/summarize"
headers = {
    "X-RapidAPI-Host": "text-monkey-summarization.p.rapidapi.com",
    "X-RapidAPI-Key": "YOUR_RAPIDAPI_KEY",
    "Content-Type": "application/json",
    "Accept": "application/json"
}



In [41]:
# Define the input text
input_text = "In the digital era, the explosion of technology has been matched by an equally seismic shift in the ways we think and talk about it. New digital tools give rise to new terms and phrases, as well as new conceptual frameworks for understanding how these tools affect and interact with society. As we contend with novel and heightened forms of harm, words like disinformation and doxing have entered the mainstream. And concepts like digital rights and data governance have come into existence amidst the push to align the technology of today with our vision for a better future.  "

# Define the summarization parameters
params = {
    "text": input_text,
    "sent_count": 3,   # Number of sentences in the summary
    "percentage": 30  # Percentage of the original text to include in the summary
}

# Send the API request
response = requests.post(url, headers=headers, data=json.dumps(params))

# Extract the summary from the response
summary = response.json()["summary"]

# Print the summary
print(summary)


KeyError: ignored

In [43]:
import requests

# Set up the API endpoint
endpoint = "https://api.textgears.com/summarize"

# Set up the parameters for the API request
params = {
    "text": "This is the text to be summarized.",
    "sentences_number": 3,
    "key": "YOUR_API_KEY"
}

# Send the API request
response = requests.get(endpoint, params=params)




KeyError: ignored

In [ ]:
# Extract the summary from the response
summary = response.json()["result"]

# Print the summary
print(summary)

In [ ]:
# trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [ ]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')

In [ ]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

['Model_Epoch_5_3000.joblib']

In [ ]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


['tokenizer_5_3000.joblib']

In [ ]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

In [ ]:
loaded_model_joblib

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')

In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = loaded_model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))